In [40]:
from time import strftime, strptime, localtime
from datetime import datetime as dt, timedelta

In [90]:
date1 = '10-5-2018'
date2 = '2017-08-10'

## A una fecha pasada por string, añadir horas, minutos y segundos

In [86]:
td1 = timedelta(hours=23, minutes=59, seconds=59)

In [91]:
dt1 = dt.strptime(date1, '%d-%m-%Y')

In [92]:
dt1

datetime.datetime(2018, 5, 10, 0, 0)

## Transformar esa fecha y hora a epoch en milisegundos (1e3)

In [93]:
int(dt.timestamp(dt1) * 1e3)

1525903200000

In [48]:
from riotwatcher import RiotWatcher
from config.constants import API_KEY

In [49]:
rw = RiotWatcher(API_KEY)

In [59]:
kwargs = {'region': 'EUW1', 'begin_index': None, 'end_index': None}

In [64]:
{k: kwargs[k] for k in ['region']}

{'region': 'EUW1'}

In [63]:
rw.match.matchlist_by_account(account_id=22404391, **kwargs)

{'endIndex': 100,
 'matches': [{'champion': 25,
   'gameId': 3631850114,
   'lane': 'BOTTOM',
   'platformId': 'EUW1',
   'queue': 420,
   'role': 'DUO_SUPPORT',
   'season': 11,
   'timestamp': 1526334404316},
  {'champion': 497,
   'gameId': 3630151834,
   'lane': 'BOTTOM',
   'platformId': 'EUW1',
   'queue': 420,
   'role': 'SOLO',
   'season': 11,
   'timestamp': 1526206954105},
  {'champion': 223,
   'gameId': 3630003381,
   'lane': 'BOTTOM',
   'platformId': 'EUW1',
   'queue': 420,
   'role': 'DUO_SUPPORT',
   'season': 11,
   'timestamp': 1526201096119},
  {'champion': 432,
   'gameId': 3629601948,
   'lane': 'BOTTOM',
   'platformId': 'EUW1',
   'queue': 420,
   'role': 'DUO_SUPPORT',
   'season': 11,
   'timestamp': 1526151741345},
  {'champion': 53,
   'gameId': 3629571127,
   'lane': 'BOTTOM',
   'platformId': 'EUW1',
   'queue': 420,
   'role': 'DUO_SUPPORT',
   'season': 11,
   'timestamp': 1526149838674},
  {'champion': 89,
   'gameId': 3627051057,
   'lane': 'BOTTOM',


In [ ]:
strftime('%d-%m-%Y', strptime(date1, '%Y-%m-%d'))

In [ ]:
strptime(date1, '%Y-%m-%d') > strptime(date2, '%Y-%m-%d')

In [ ]:
strptime(date1, '%Y-%m-%d') < strptime(date2, '%Y-%m-%d')

In [ ]:
date1 + date2

In [ ]:
date1 < date2

In [2]:
dt(2018, 12, 23)

datetime.datetime(2018, 12, 23, 0, 0)

In [ ]:
dt.fromtimestamp(int(dt.timestamp(dt(2018, 12, 22, 23, 59, 59)))).strftime('%Y-%m-%d %H:%M:%S')

In [81]:
strftime('%Y-%m-%d', localtime(1524686549464/1e3))

'2018-04-25'

# Reading DFs

In [3]:
import pandas as pd

In [4]:
from config.constants import LEAGUES_DATA_DICT, EXCEL_EXPORT_PATH_MERGED, EXCEL_EXPORT_PATH, SQL_LEAGUES_ENGINE
from sqlalchemy import create_engine

In [5]:
df = pd.read_excel(LEAGUES_DATA_DICT['SOLOQ'][EXCEL_EXPORT_PATH_MERGED])

In [14]:
df1 = pd.read_excel(LEAGUES_DATA_DICT['SOLOQ'][EXCEL_EXPORT_PATH])

In [10]:
len(df.gameId.unique()), df.shape[0]

(17, 171)

In [17]:
df.groupby('gameId').first().gameVersion.value_counts()

8.1.214.5847    12
8.1.213.4336     5
Name: gameVersion, dtype: int64